# Test a New Wing Shape — Without Spinning Up the nTop GUI

**Use case:** "I want to change my wing parameters and see how it performs."

This notebook connects to Istari, navigates the system hierarchy to find a parametric nTop model,
submits a `run_model` job with configurable wing parameters, snapshots the results, then displays
aerodynamic metrics and rendered views — all while Istari tracks every version and milestone.

**What you'll get:**
- A walkthrough of Istari's version control: systems, configurations, snapshots, and tags
- Aerodeck performance metrics (structure weight, range, cruise speed)
- 7 rendered views of the wing geometry (iso, top, front, back, left, right, bottom)
- Updated .ntop model and .obj mesh as downloadable artifacts
- Tagged snapshots capturing before and after the design run

**Example model:** Group 3 UAS Wing v8 — a parametric tailless flying wing.

See [`example-output/`](example-output/) for what the results look like without running anything.

In [ ]:
# Setup — install SDK and connect to Istari
import sys, os
from pathlib import Path

try:
    import istari_digital_client
except ImportError:
    !pip install istari-digital-client python-dotenv -q

repo_root = str(Path.cwd().parent.parent)
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from istari_client import get_client

client = get_client()
user = client.get_current_user()
print(f"Connected as: {user.display_name} ({user.email})")

In [ ]:
# Explore the system — navigate the Istari hierarchy
#
# Instead of jumping straight to a file, start from the system level.
# Istari organizes everything as: System -> Configuration -> Snapshot -> Files
#
# Istari links:
#   System: https://demo.istari.app/systems/91cde24e-8343-44ba-8b8f-67dc6e9e5334
#   Model:  https://demo.istari.app/files/263b7332-03f4-4ded-9686-7f11df478058

SYSTEM_ID = "91cde24e-8343-44ba-8b8f-67dc6e9e5334"  # Example: nTop Wing Design
CONFIG_ID = "2fa8789e-ae4b-4065-85fc-f979a817b80d"  # Baseline configuration

system = client.get_system(SYSTEM_ID)
print(f"System: {system.name}")
print(f"  {system.description}\n")

# Walk the hierarchy: configurations -> tracked files -> snapshots -> files
configs = client.list_system_configurations(SYSTEM_ID, page=1, size=50)
print(f"Configurations ({configs.total}):\n")

for config in configs.items:
    print(f"  {config.name}")
    print(f"    Config ID: {config.id}")

    # What files does this configuration track?
    tracked = client.list_tracked_files(config.id, page=1, size=50)
    for tf in tracked.items:
        mode = tf.specifier_type.value  # "Latest" or "Locked"
        print(f"    Tracked file: {tf.file_id} ({mode})")

    # What snapshots exist? (point-in-time captures)
    snapshots = client.list_snapshots(configuration_id=config.id, page=1, size=10)
    print(f"    Snapshots ({snapshots.total}):")

    for snap in snapshots.items:
        tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
        tag_names = [t.tag for t in tags.items]
        tag_str = f"  [{', '.join(tag_names)}]" if tag_names else ""
        revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
        print(f"      {snap.id[:8]}...{tag_str}  ({revs.total} file(s))")
        for r in revs.items:
            size_kb = r.size / 1024
            if size_kb > 1024:
                print(f"        - {r.name} ({size_kb/1024:.1f} MB)")
            else:
                print(f"        - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# Configure the model and parameters
#
# Replace MODEL_ID with your own nTop model, or use the default Group3 UAS wing.
# Modify the parameter values below to explore different wing configurations.

MODEL_ID = "263b7332-03f4-4ded-9686-7f11df478058"  # Group3-UAS-Wing-v8

# Wing design parameters — edit these to try different configurations
LOA_IN = 99.9          # Overall length (inches)
SPAN = 144             # Wingspan (inches)
LE_SWEEP_P1 = 46       # Leading edge sweep, inboard (degrees)
LE_SWEEP_P2 = 46       # Leading edge sweep, outboard (degrees)
TE_SWEEP_P1 = -46      # Trailing edge sweep, inboard (degrees)
TE_SWEEP_P2 = 15       # Trailing edge sweep, outboard (degrees)
PANEL_BREAK = 0.30     # Panel break spanwise location (fraction)

# Build the input JSON
input_data = {
    "inputs": [
        {"name": "LOA In", "type": "real", "units": "in", "value": LOA_IN},
        {"name": "Span", "type": "real", "units": "in", "value": SPAN},
        {"name": "LE Sweep P1", "type": "real", "units": "deg", "value": LE_SWEEP_P1},
        {"name": "LE Sweep P2", "type": "real", "units": "deg", "value": LE_SWEEP_P2},
        {"name": "TE Sweep P1", "type": "real", "units": "deg", "value": TE_SWEEP_P1},
        {"name": "TE Sweep P2", "type": "real", "units": "deg", "value": TE_SWEEP_P2},
        {"name": "Panel Break Span %", "type": "real", "value": PANEL_BREAK},
        {"name": "MAIN PATH", "type": "file_path", "value": "/home/bradrothenberg/nTopGrp3/output/"},
    ]
}

print("Wing parameters:")
for inp in input_data["inputs"]:
    if inp["type"] != "file_path":
        unit = inp.get("units", "")
        print(f"  {inp['name']}: {inp['value']} {unit}")

# Check current model state and revision history
model = client.get_model(MODEL_ID)
print(f"\nModel: {model.display_name or 'Group3-UAS-Wing-v8'}")
print(f"  File ID: {model.file.id}")
print(f"  Revisions: {len(model.file.revisions)}")
for rev in model.file.revisions:
    size_mb = rev.size / (1024 * 1024)
    print(f"    - {rev.name} ({size_mb:.1f} MB)")
print(f"  Existing artifacts: {len(model.artifacts)}")

# Show how the model connects to the system hierarchy
print(f"\nThis file is tracked in the '{configs.items[0].name}' configuration")
print(f"as a LATEST tracked file — any new revisions will be automatically picked up.")

In [ ]:
# Submit the nTop job
#
# This runs ntopcl v5.30 on RHEL 8 with the parameters above.
# Typical run time: 3-8 minutes depending on agent availability.

from time import sleep
from datetime import datetime
from istari_digital_client import JobStatusName

print("Submitting @ntop:run_model job...")
job = client.add_job(
    model_id=MODEL_ID,
    function="@ntop:run_model",
    tool_name="ntopcl",
    tool_version="5.30",
    operating_system="RHEL 8",
    parameters={"ntop_input_json": input_data},
)
print(f"Job: {job.id}")

# Poll until done
while True:
    sleep(10)
    job = client.get_job(job.id)
    ts = datetime.now().strftime("%H:%M:%S")
    status = job.status.name.value
    print(f"  [{ts}] {status}")
    if job.status.name in {JobStatusName.COMPLETED, JobStatusName.FAILED}:
        break

if job.status.name == JobStatusName.COMPLETED:
    print("\nJob completed!")
    model = client.get_model(MODEL_ID)  # refresh
    print(f"Total artifacts on model: {len(model.artifacts)}")
else:
    print("\nJob FAILED!")
    if hasattr(job, 'status_history') and job.status_history:
        for s in job.status_history:
            print(f"  {s.name}: {getattr(s, 'message', '')}")

In [ ]:
# Snapshot the run results
#
# A snapshot captures the exact state of all tracked files at this moment.
# Think of it like a git tag — an immutable bookmark you can always return to.
#
# Before the nTop run: 1 file  (just the .ntop model)
# After the nTop run:  15 files (.ntop + 14 artifacts: mesh, metrics, views)
#
# Creating a snapshot here means anyone can later see exactly what parameters
# produced what results — and compare across design iterations.

from istari_digital_client import NewSnapshot, NewSnapshotTag

print("Creating post-run snapshot...")
snap_response = client.create_snapshot(CONFIG_ID, NewSnapshot())
snapshot = snap_response.actual_instance

# Handle both new snapshot and no-op (if snapshot already exists for this state)
if hasattr(snapshot, "id"):
    snapshot_id = snapshot.id
    print(f"Snapshot created: {snapshot_id}")
else:
    # No changes since last snapshot — find the most recent one
    snaps = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=1)
    snapshot_id = snaps.items[0].id
    print(f"No changes since last snapshot — using: {snapshot_id}")

# Tag it with a human-readable name
tag = client.create_tag(snapshot_id, NewSnapshotTag(tag="post-ntop-run"))
print(f"Tagged as: '{tag.tag}'")

# Verify what's in the snapshot
revs = client.list_snapshot_revisions(snapshot_id, page=1, size=50)
print(f"\nSnapshot contains {revs.total} file(s):")
for r in revs.items:
    size_kb = r.size / 1024
    if size_kb > 1024:
        print(f"  - {r.name} ({size_kb/1024:.1f} MB)")
    else:
        print(f"  - {r.name} ({size_kb:.1f} KB)")

In [ ]:
# View aerodeck metrics
#
# The aerodeck_metrics.json contains the key performance numbers:
# structure weight, range, cruise speed, and more.

import json
from IPython.display import HTML, display

# Refresh model
model = client.get_model(MODEL_ID)

# Find the most recent aerodeck_metrics artifact
metrics_artifact = None
for a in reversed(model.artifacts):
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and "aerodeck_metrics" in rev.name:
        metrics_artifact = a
        break

if metrics_artifact:
    metrics = json.loads(metrics_artifact.read_text())
    print("Aerodeck Metrics:\n")
    print(json.dumps(metrics, indent=2))
    
    # Extract key values for a summary
    range_data = metrics.get("range_mission", {})
    weight_data = metrics.get("weight", metrics)
    
    summary_html = """
    <table style="border-collapse: collapse; font-size: 14px; margin-top: 16px;">
    <thead>
        <tr style="background: #f1f5f9;">
            <th style="padding: 10px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Metric</th>
            <th style="padding: 10px 16px; text-align: left; border-bottom: 2px solid #cbd5e1;">Value</th>
        </tr>
    </thead>
    <tbody>
    """
    for key, val in metrics.items():
        if isinstance(val, dict):
            for k2, v2 in val.items():
                summary_html += f'<tr><td style="padding: 6px 16px;">{key} / {k2}</td><td style="padding: 6px 16px;"><b>{v2}</b></td></tr>\n'
        else:
            summary_html += f'<tr><td style="padding: 6px 16px;">{key}</td><td style="padding: 6px 16px;"><b>{val}</b></td></tr>\n'
    summary_html += "</tbody></table>"
    display(HTML(summary_html))
else:
    print("Aerodeck metrics not found. Run the job cell first.")

In [ ]:
# View rendered wing images
#
# Each nTop run produces 7 PNG views: top, front, back, left, right, bottom, iso.
# We'll display the most recent set.

from IPython.display import Image, display, Markdown

# Find the most recent set of view PNGs
view_names = ["iso", "top", "front", "right"]
view_artifacts = {}

for a in reversed(model.artifacts):
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev and rev.name.endswith(".png"):
        base = rev.name.replace(".png", "")
        if base in view_names and base not in view_artifacts:
            view_artifacts[base] = a
    if len(view_artifacts) == len(view_names):
        break

if view_artifacts:
    display(Markdown("### Wing Views"))
    for name in view_names:
        if name in view_artifacts:
            display(Markdown(f"**{name.capitalize()}**"))
            img_bytes = view_artifacts[name].read_bytes()
            display(Image(data=img_bytes, width=500))
else:
    print("View PNGs not found. Run the job cell first.")

In [ ]:
# List all artifacts from this run
#
# The full list includes the updated .ntop model, .obj mesh, aerodeck data,
# and all rendered views.

print(f"All artifacts on model ({len(model.artifacts)} total):\n")
for a in model.artifacts:
    rev = a.file.revisions[0] if a.file.revisions else None
    if rev:
        size_kb = rev.size / 1024
        if size_kb > 1024:
            size_str = f"{size_kb/1024:.1f} MB"
        else:
            size_str = f"{size_kb:.1f} KB"
        print(f"  {rev.name:40s}  {size_str:>10s}")

In [ ]:
# View the full system state — everything Istari tracked
#
# This is the "outer loop" view. The system now has tagged snapshots showing:
#   1. "initial-upload" — just the .ntop model before any runs
#   2. "post-ntop-run"  — .ntop + 14 artifacts from the design run
#
# Anyone on the team can see exactly what changed and when.
# Run this cell again after trying different parameters to see the history grow.

print(f"System: {system.name}\n")

all_snapshots = client.list_snapshots(configuration_id=CONFIG_ID, page=1, size=20)
print(f"Total snapshots: {all_snapshots.total}\n")

for i, snap in enumerate(all_snapshots.items, 1):
    tags = client.list_tags(snapshot_id=snap.id, page=1, size=10)
    tag_names = [t.tag for t in tags.items]
    tag_str = ", ".join(tag_names) if tag_names else "untagged"
    
    revs = client.list_snapshot_revisions(snap.id, page=1, size=50)
    
    print(f"  Snapshot {i}: [{tag_str}]")
    print(f"    Created: {snap.created}")
    print(f"    Files ({revs.total}):")
    for r in revs.items:
        size_kb = r.size / 1024
        if size_kb > 1024:
            size_str = f"{size_kb/1024:.1f} MB"
        else:
            size_str = f"{size_kb:.1f} KB"
        print(f"      - {r.name:40s} {size_str:>10s}")
    print()

print("Every file, every design iteration, every milestone — tracked in one place.")

## Summary

Each nTop run produces ~14 artifacts from a single set of wing parameters:

| Artifact Type | Files | Purpose |
|--------------|-------|----------|
| Updated .ntop model | 1 | Geometry with computed parameters baked in |
| .obj mesh | 1 | 3D mesh for visualization or downstream analysis |
| Aerodeck metrics | 3 (JSON, HTML, full JSON) | Performance: weight, range, cruise speed |
| Output summary | 2 (output.json, _output.json) | Run metadata |
| View PNGs | 7 | top, front, back, left, right, bottom, isometric |

### Version control in action

Istari tracked the entire design iteration automatically:

| What Istari Did | How |
|----------------|-----|
| Stored the .ntop model | `add_model()` → file revision 1 |
| Organized it in a configuration | "Baseline" config with LATEST tracking |
| Bookmarked the upload | Snapshot tagged "initial-upload" |
| Ran nTop with wing parameters | `add_job()` → 14 artifacts produced |
| Bookmarked the run | Snapshot tagged "post-ntop-run" |

Change the parameters and run again → new artifacts, new snapshot, new bookmark.
Each design iteration is captured and comparable. This is the **outer loop** — the version
control layer that sits above any individual tool.

## What's Next?

- **Try different parameters** — change the values in cell 3 and re-run to explore the design space
- **Compare design iterations** — look at snapshots side-by-side in the Istari UI
- **Check against requirements** — compare metrics to the SysML requirements in [`use-cases/explore-sysml-model/`](../explore-sysml-model/)
- **Promote a run as a new version** — see [`getting-started/02_version_model.py`](../../getting-started/02_version_model.py)
- **Share with your team** — see [`getting-started/03_share_resources.py`](../../getting-started/03_share_resources.py)